In [89]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [90]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [91]:
import io
df = pd.read_csv('for_model.csv')


In [92]:
df.head()

,Review,Rating
0,"The ambience was good, food was quite good . h...",5.0
1,Ambience is too good for a pleasant evening. S...,5.0
2,A must try.. great food great ambience. Thnx f...,5.0
3,Soumen das and Arun was a great guy. Only beca...,5.0
4,Food is good.we ordered Kodi drumsticks and ba...,5.0


In [93]:
df.shape

(9955, 2)

In [94]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9955 entries, 0 to 9954
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Review  9955 non-null   object 
 1   Rating  9955 non-null   float64
dtypes: float64(1), object(1)
memory usage: 155.7+ KB


In [96]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\hp-
[nltk_data]     pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [97]:
y=pd.get_dummies(df['Rating'])

In [112]:
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['Review'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [113]:
corpus

['ambience good food quite good saturday lunch cost effective good place sate brunch one also chill friend parent waiter soumen da really courteous helpful',
 'ambience good pleasant evening service prompt food good good experience soumen da kudos service',
 'must try great food great ambience thnx service pradeep subroto personal recommendation penne alfredo pasta also music background amazing',
 'soumen da arun great guy behavior sincerety good food course would like visit place',
 'food good ordered kodi drumstick basket mutton biryani good thanks pradeep served well enjoyed ambience also good',
 'ambiance good service good food apradeecp subro best service food good papiya good hostess ur caption good star restaurant',
 'nice place ambience different food ordered tasty service also gud worth visit reasonable well really must visit place',
 'well reading many review finally visited place ambience good coming food crispy corn nice tawa fish ok basket biryani disappointed u biryani ok

In [114]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [115]:
voc_size=10000


In [116]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4136,
  7066,
  1610,
  9800,
  7066,
  7315,
  5988,
  2064,
  7807,
  7066,
  2860,
  501,
  7453,
  3478,
  6554,
  4907,
  2714,
  2228,
  8421,
  6811,
  2359,
  3310,
  7645,
  2990],
 [4136,
  7066,
  6948,
  5393,
  4825,
  7482,
  1610,
  7066,
  7066,
  8017,
  6811,
  2359,
  6932,
  4825],
 [765,
  5374,
  6375,
  1610,
  6375,
  4136,
  4939,
  4825,
  9854,
  3339,
  1051,
  529,
  8551,
  9280,
  7292,
  6554,
  8385,
  6331,
  4668],
 [6811,
  2359,
  466,
  6375,
  6267,
  5086,
  6591,
  7066,
  1610,
  1398,
  3849,
  2676,
  1560,
  2860],
 [1610,
  7066,
  880,
  6140,
  9782,
  6149,
  8891,
  6504,
  7066,
  269,
  9854,
  9287,
  7677,
  1694,
  4136,
  6554,
  7066],
 [3147,
  7066,
  4825,
  7066,
  1610,
  7587,
  1880,
  5856,
  4825,
  1610,
  7066,
  8492,
  7066,
  8278,
  1896,
  4165,
  7066,
  6979,
  1050],
 [2939,
  2860,
  4136,
  6255,
  1610,
  880,
  6742,
  4825,
  6554,
  8232,
  715,
  1560,
  7657,
  7677,
  3310,
  765,
  1560,
  2860],
 [

In [117]:
sent_length=50
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3310 7645 2990]
 [   0    0    0 ... 2359 6932 4825]
 [   0    0    0 ... 8385 6331 4668]
 ...
 [3122 8017 6357 ... 4009 8065 3565]
 [   0    0    0 ... 5317 4825 1817]
 [5178 3688 9116 ... 3726 4668 4136]]


In [121]:
def create_model():
 
 
 
 
 embedding_vector_features=40
 model=Sequential()
 model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
 model.add(LSTM(200,recurrent_dropout=0.1))
 model.add(Dropout(0.3))
 model.add(Dense(5,activation='softmax'))


 model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
 return model
#print(model.summary())

In [122]:
model = create_model()


In [123]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 40)            400000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 200)               192800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 1005      
Total params: 593,805
Trainable params: 593,805
Non-trainable params: 0
_________________________________________________________________


In [124]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)


In [125]:
X_final.shape,y_final.shape


((9955, 50), (9955, 5))

In [126]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.10, random_state=42)

In [127]:
y_test

array([[0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]], dtype=uint8)

In [128]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
140/140 [==============================] - 34s 246ms/step - loss: 1.2949 - accuracy: 0.4617 - val_loss: 1.0247 - val_accuracy: 0.5703
Epoch 2/10
140/140 [==============================] - 31s 224ms/step - loss: 0.9187 - accuracy: 0.6211 - val_loss: 0.9325 - val_accuracy: 0.6084
Epoch 3/10
140/140 [==============================] - 31s 218ms/step - loss: 0.7876 - accuracy: 0.6827 - val_loss: 0.9676 - val_accuracy: 0.5833
Epoch 4/10
140/140 [==============================] - 30s 216ms/step - loss: 0.7028 - accuracy: 0.7234 - val_loss: 1.0251 - val_accuracy: 0.5823
Epoch 5/10
140/140 [==============================] - 30s 217ms/step - loss: 0.6241 - accuracy: 0.7590 - val_loss: 1.0617 - val_accuracy: 0.5863
Epoch 6/10
140/140 [==============================] - 30s 215ms/step - loss: 0.5692 - accuracy: 0.7887 - val_loss: 1.1661 - val_accuracy: 0.5863
Epoch 7/10
140/140 [==============================] - 30s 213ms/step - loss: 0.5152 - accuracy: 0.8118 - val_loss: 1.2023 - val_ac

In [129]:
y_pred1=model.predict_classes(X_test)


In [130]:
y_pred1

array([4, 2, 0, 4, 3, 4, 0, 3, 3, 4, 3, 0, 1, 3, 4, 4, 2, 4, 0, 0, 4, 4,
       1, 4, 1, 3, 2, 2, 4, 4, 4, 3, 4, 3, 4, 0, 4, 4, 3, 3, 3, 2, 3, 4,
       4, 4, 3, 3, 3, 3, 4, 4, 4, 0, 3, 0, 0, 3, 0, 3, 4, 4, 3, 3, 1, 0,
       0, 3, 0, 4, 4, 4, 0, 4, 4, 4, 2, 0, 1, 3, 0, 3, 4, 4, 3, 0, 4, 2,
       4, 0, 4, 0, 0, 4, 4, 4, 4, 0, 1, 0, 2, 4, 4, 4, 4, 2, 4, 2, 3, 2,
       3, 4, 0, 4, 4, 2, 4, 4, 4, 2, 0, 0, 4, 4, 0, 4, 2, 4, 0, 3, 0, 4,
       0, 4, 4, 4, 0, 0, 4, 3, 4, 4, 2, 0, 2, 2, 0, 4, 2, 0, 4, 0, 4, 0,
       4, 0, 3, 2, 4, 2, 4, 3, 3, 4, 4, 4, 0, 2, 1, 4, 4, 0, 4, 0, 3, 3,
       4, 0, 4, 3, 0, 0, 4, 4, 3, 3, 2, 3, 3, 0, 3, 0, 3, 4, 3, 3, 4, 4,
       4, 1, 2, 0, 4, 3, 4, 4, 3, 3, 0, 3, 3, 2, 3, 2, 1, 4, 4, 0, 3, 4,
       3, 4, 4, 4, 4, 0, 3, 3, 2, 0, 3, 1, 0, 2, 4, 4, 3, 2, 4, 1, 4, 3,
       4, 4, 4, 4, 4, 4, 1, 3, 4, 4, 0, 0, 4, 4, 4, 4, 4, 2, 2, 2, 3, 3,
       0, 3, 0, 4, 4, 4, 3, 0, 4, 0, 0, 0, 0, 0, 4, 4, 2, 4, 4, 0, 4, 1,
       4, 4, 3, 4, 0, 4, 0, 4, 0, 4, 3, 0, 3, 0, 3,

In [199]:
pip install git+git://github.com/keras-team/keras.git --upgrade --no-deps

  Cloning git://github.com/keras-team/keras.git to /tmp/pip-req-build-85f_tpe4
  Running command git clone -q git://github.com/keras-team/keras.git /tmp/pip-req-build-85f_tpe4
  Created wheel for Keras: filename=Keras-2.4.2-cp36-none-any.whl size=36257 sha256=e18fc8e335d7f5b855e6f3c505072c0dd7eeb44fba4a7b39234230699db7cf29
  Stored in directory: /tmp/pip-ephem-wheel-cache-_oot5yps/wheels/e6/02/ad/5e8e1a5824af71082e2260fe7e2eaa1b745c34706e6ff0a14b
Successfully built Keras
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [132]:
from tensorflow.keras.models import load_model


In [133]:
model.save('lstm_model.h5')

In [134]:
k = load_model('lstm_model.h5')
# make predictions
yhat = k.predict_classes(X_test)
print(yhat)

[4 2 0 4 3 4 0 3 3 4 3 0 1 3 4 4 2 4 0 0 4 4 1 4 1 3 2 2 4 4 4 3 4 3 4 0 4
 4 3 3 3 2 3 4 4 4 3 3 3 3 4 4 4 0 3 0 0 3 0 3 4 4 3 3 1 0 0 3 0 4 4 4 0 4
 4 4 2 0 1 3 0 3 4 4 3 0 4 2 4 0 4 0 0 4 4 4 4 0 1 0 2 4 4 4 4 2 4 2 3 2 3
 4 0 4 4 2 4 4 4 2 0 0 4 4 0 4 2 4 0 3 0 4 0 4 4 4 0 0 4 3 4 4 2 0 2 2 0 4
 2 0 4 0 4 0 4 0 3 2 4 2 4 3 3 4 4 4 0 2 1 4 4 0 4 0 3 3 4 0 4 3 0 0 4 4 3
 3 2 3 3 0 3 0 3 4 3 3 4 4 4 1 2 0 4 3 4 4 3 3 0 3 3 2 3 2 1 4 4 0 3 4 3 4
 4 4 4 0 3 3 2 0 3 1 0 2 4 4 3 2 4 1 4 3 4 4 4 4 4 4 1 3 4 4 0 0 4 4 4 4 4
 2 2 2 3 3 0 3 0 4 4 4 3 0 4 0 0 0 0 0 4 4 2 4 4 0 4 1 4 4 3 4 0 4 0 4 0 4
 3 0 3 0 3 2 3 2 4 0 3 3 4 3 0 3 0 0 3 4 4 3 4 0 4 4 0 4 4 4 4 4 4 4 4 3 0
 0 4 0 2 4 4 4 3 3 4 4 4 1 4 4 4 4 3 4 3 0 0 4 2 2 0 3 2 0 3 0 4 4 4 0 3 3
 4 0 3 4 3 4 1 3 4 3 4 4 0 4 4 2 0 3 3 4 4 4 4 0 4 3 4 4 2 4 1 4 4 4 4 4 4
 4 4 3 2 3 4 3 3 4 4 2 4 0 4 3 4 4 3 3 4 4 4 0 4 4 2 4 3 3 1 4 3 3 0 0 4 3
 0 4 4 4 4 3 2 1 4 0 4 1 0 4 4 0 4 4 4 2 3 0 1 4 4 3 0 3 4 3 4 0 3 0 4 4 2
 4 4 4 4 3 4 4 4 4 4 4 4 